In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import logging
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Import from scripts
sys.path.append("..") 
from src.data_loader import load_tsla_data
from scripts.lstm import (
    preprocess_data,
    build_lstm_model,
    train_lstm,
    forecast_lstm,
    evaluate_lstm,
    plot_lstm_results
)
                             

In [3]:
data = load_tsla_data('../data/TSLA_cleaned.csv')  



2025-03-01 23:23:35,109 - INFO - Loading TSLA data from CSV...
2025-03-01 23:23:35,133 - INFO - Verifying TSLA data integrity...
2025-03-01 23:23:35,136 - INFO - TSLA data loaded successfully. Shape: (2505, 9), Missing values: 0


In [4]:
data = data.asfreq('B').ffill()
data.head() 

,Price,Close,High,Low,Volume,Daily_Return,Rolling_Mean,Rolling_Std,Z_Score
Date,,,,,,,,,
2015-02-17,13.623333,13.713333,13.433333,13.713333,59694000.0,-0.140787,13.940933,0.583636,-0.895868
2015-02-18,13.630667,13.744667,13.506667,13.611333,40704000.0,0.228492,13.917978,0.577107,-0.895605
2015-02-19,14.114000,14.162667,13.583333,13.666667,77311500.0,3.041181,13.914067,0.574963,-0.892099
2015-02-20,14.474000,14.506667,13.987333,14.052000,89731500.0,2.428920,13.920333,0.580523,-0.889213
2015-02-23,13.822667,14.546667,13.755333,14.377333,127497000.0,0.275735,13.930111,0.588737,-0.888878


In [5]:
# Preprocess data
X_train, y_train, X_test, y_test, scaler = preprocess_data(data, look_back=60)
print("\nPreprocessed Data Shapes:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

2025-03-01 23:23:52,534 - INFO - Starting data preprocessing
2025-03-01 23:23:52,574 - INFO - Original data range: 2015-02-17 00:00:00 to 2025-01-30 00:00:00
2025-03-01 23:23:52,587 - INFO - Training sequences: 2018
2025-03-01 23:23:52,588 - INFO - Test sequences: 460



Preprocessed Data Shapes:
X_train: (2018, 60, 1), y_train: (2018, 1)
X_test: (460, 60, 1), y_test: (460, 1)


In [8]:
# Build model
model = build_lstm_model(input_shape=(X_train.shape[1], 1))

2025-03-01 23:24:22,249 - INFO - Building LSTM model


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 60, 128)        │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 60, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,081 (461.25 KB)

 Trainable params: 118,081 (461.25 KB)

 Non-trainable params: 0 (0.00 B)

2025-03-01 23:24:22,321 - INFO - Model summary:
None


In [9]:
# Train model
trained_model, training_history = train_lstm(model, X_train, y_train)

2025-03-01 23:24:57,759 - INFO - Starting model training


Epoch 1/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 139ms/step - loss: 0.0282 - mae: 0.1210 - val_loss: 0.0363 - val_mae: 0.1526 - learning_rate: 0.0010
Epoch 2/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 111ms/step - loss: 0.0293 - mae: 0.1287 - val_loss: 0.0320 - val_mae: 0.1406 - learning_rate: 0.0010
Epoch 3/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 118ms/step - loss: 0.0252 - mae: 0.1161 - val_loss: 0.0295 - val_mae: 0.1331 - learning_rate: 0.0010
Epoch 4/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - loss: 0.0237 - mae: 0.1068 - val_loss: 0.0283 - val_mae: 0.1286 - learning_rate: 0.0010
Epoch 5/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - loss: 0.0242 - mae: 0.1053 - val_loss: 0.0277 - val_mae: 0.1261 - learning_rate: 0.0010
Epoch 6/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step - loss: 0.0200 - mae: 0.0976 - val_loss: 0.0275 - val_mae: 0.1249 - learning_rate: 0.0010
Epoch 7/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 115ms/step - loss: 0.0221 - mae: 0.0999 - val_loss: 0.0273 - val_mae: 0.1240 - learning_rate: 0.0010

KeyboardInterrupt: 

In [ ]:
# Generate forecasts
predicted_prices = forecast_lstm(trained_model, X_test, scaler, data)



In [ ]:
# Prepare test data
actual_prices = data['Close'].iloc[-len(predicted_prices):].values
dates = data.index[-len(predicted_prices):]